# NOTES

- Applying `normalize_subset` to all records that contain `"(?i)microsoft|msft"` - i.e., a group of records that almost certainly all belong to MSFT - the algorithm works. This implies that the issue is indeed with the subgraph creation. I.e., the `"microsoft"` subgraph is being linked to other companies. Most likely, this is because there are 1 or 2 records with, e.g., `{"issuer_name": "microsoft", "cusip": apple's cusip}`. We 

# Main

In [16]:
import polars as pl
from record_consolidation.graphs import (
    unconsolidated_df_to_subgraphs,
    _extract_canonicals_from_subgraph,
)
from record_consolidation.subgraph_post_processing.specific_algs.split_high_betweenness import (
    split_subgraph_where_necessary,
)
import networkx as nx

In [2]:
# votes: pl.DataFrame = access_db_table("raw_output", "votes")
votes: pl.DataFrame = pl.read_parquet("raw_votes.parquet")
COMPANY_COLS = pl.col(["issuer_name", "cusip", "isin", "figi"])
MEETING_COLS = pl.col(["issuer_name", "meeting_date", "cusip", "isin", "figi"])

In [3]:
def ps(subgraph: nx.Graph) -> None:
    for x in subgraph.nodes.data():
        print(x)

In [4]:
from typing import Iterable
from warnings import warn


def extract_specific_name_subgraph(
    connected_subgs: Iterable[nx.Graph], name: str
) -> nx.Graph:
    collected: list[nx.Graph] = []
    for subg in connected_subgs:
        for n in subg.nodes.data():
            if name.lower() in (n[0]).lower():
                collected.append(subg)
    collected = set(collected)
    if len(collected) == 0:
        raise ValueError(f"Could not find {name=} in connected_subgs.")
    if len(collected) > 1:
        alert_str = f"{len(collected)} subgraphs have {name=}.\n{[len(subg.nodes) for subg in collected]=}"
        warn(alert_str)
        return max(collected, key=lambda g: len(g.nodes))

        # raise ValueError(alert_str)
    return tuple(collected)[0]

# Extract SubGraphs

## Post Processing

In [ ]:
post_processed_subgraphs = list(
    unconsolidated_df_to_subgraphs(
        votes.select(COMPANY_COLS),  # .head(100),
        weight_edges=True,
        connected_subgraphs_postprocessor=split_subgraph_where_necessary,
    )
)

In [10]:
cvx = extract_specific_name_subgraph(post_processed_subgraphs, "CHEVRON")

/var/folders/z7/gyrmrn1s10zf6k523_6hg_br0000gn/T/ipykernel_40657/942150125.py:18: UserWarning: 2 subgraphs have name='CHEVRON'.
[len(subg.nodes) for subg in collected]=[17, 1]
  warn(alert_str)


In [13]:
ps(cvx)

('Chevron', {'field': 'issuer_name', 'count': 1})
('Chevron Corposration', {'field': 'issuer_name', 'count': 1})
('CHEVRON CORP', {'field': 'issuer_name', 'count': 21})
('Chevron Corp.', {'field': 'issuer_name', 'count': 4})
('Chevron Corpoation', {'field': 'issuer_name', 'count': 1})
('CHEVRON CORP NEW COM', {'field': 'issuer_name', 'count': 1})
('CHEVRON', {'field': 'issuer_name', 'count': 2})
('BBG000K4ND22', {'field': 'figi', 'count': 5})
('Chevron Corp (CVX)', {'field': 'issuer_name', 'count': 1})
('Chrvron Corporation', {'field': 'issuer_name', 'count': 1})
('US1667641005', {'field': 'isin', 'count': 1227})
('CHEVRON CORPORATION', {'field': 'issuer_name', 'count': 1034})
('CHEVRON CORP NEW', {'field': 'issuer_name', 'count': 2})
('CVX', {'field': 'issuer_name', 'count': 1})
('Chevron Corp', {'field': 'issuer_name', 'count': 8})
('166764100', {'field': 'cusip', 'count': 1818})
('BBG001S67ZC5', {'field': 'figi', 'count': 2})


In [12]:
msft_subgraph = extract_specific_name_subgraph(post_processed_subgraphs, "microsoft")
mcds_subgraph = extract_specific_name_subgraph(post_processed_subgraphs, "mcdonald")
jpm_subgraph = extract_specific_name_subgraph(post_processed_subgraphs, "jpmorgan")

/var/folders/z7/gyrmrn1s10zf6k523_6hg_br0000gn/T/ipykernel_35385/942150125.py:18: UserWarning: 3 subgraphs have name='mcdonald'.
[len(subg.nodes) for subg in collected]=[2, 1, 27]
  warn(alert_str)
/var/folders/z7/gyrmrn1s10zf6k523_6hg_br0000gn/T/ipykernel_35385/942150125.py:18: UserWarning: 2 subgraphs have name='jpmorgan'.
[len(subg.nodes) for subg in collected]=[1, 25]
  warn(alert_str)


In [16]:
ps(jpm_subgraph)

('JP MORGAN CHASE & CO', {'field': 'issuer_name', 'count': 1})
('JPMORGAN CHASE  CO.', {'field': 'issuer_name', 'count': 80})
('JPMorgan Chase & CO.', {'field': 'issuer_name', 'count': 20})
('JPMORGAN CHASE AND CO.', {'field': 'issuer_name', 'count': 1})
('JP MORGAN CHASE & CO.', {'field': 'issuer_name', 'count': 1})
('JPMorgan Chase', {'field': 'issuer_name', 'count': 7})
('BBG000DMBXR2', {'field': 'figi', 'count': 23})
('US46625H1005', {'field': 'isin', 'count': 1965})
('JP MORGAN CHASE & CO ', {'field': 'issuer_name', 'count': 2})
('46625h100', {'field': 'cusip', 'count': 4})
('JPMORGAN CHASE &amp; CO.', {'field': 'issuer_name', 'count': 2})
('JPMORGAN CHASE & CO.', {'field': 'issuer_name', 'count': 1324})
('JP Morgan', {'field': 'issuer_name', 'count': 1})
('JP Morgan Chase & Co', {'field': 'issuer_name', 'count': 7})
('JPMORGAN CHASE and CO.', {'field': 'issuer_name', 'count': 27})
('JPMorgan Chase & Co (JPM)', {'field': 'issuer_name', 'count': 1})
('46625H100', {'field': 'cusip',

## No PostProcessing

In [14]:
subgraphs = list(
    unconsolidated_df_to_subgraphs(
        votes.select(COMPANY_COLS),
        weight_edges=True,
        connected_subgraphs_postprocessor=None,
    )
)

## Compare the two

In [15]:
erroneously_linked_cos = [
    "microsoft",
    "chevron",
    "jpmorgan",
    "starbucks",
    "proctor",
]
for co in erroneously_linked_cos:
    post_processed = extract_specific_name_subgraph(post_processed_subgraphs, name=co)
    unprocessed = extract_specific_name_subgraph(subgraphs, name=co)

    print("-" * 120)

    print("UNPROCESSED:\n")
    ps(unprocessed)

    print("PROCESSED:\n")
    ps(post_processed)

------------------------------------------------------------------------------------------------------------------------
UNPROCESSED:

('Duke Energy', {'field': 'issuer_name', 'count': 4})
('DUKE ENERGY CORP', {'field': 'issuer_name', 'count': 1})
('GE HealthCare Technologies Inc', {'field': 'issuer_name', 'count': 1})
('MICROSOFT CORPORATION COM', {'field': 'issuer_name', 'count': 2})
('26441c204', {'field': 'cusip', 'count': 1})
('USU3644QAF29', {'field': 'isin', 'count': 1})
('Mckesson Corporation', {'field': 'issuer_name', 'count': 5})
('594918105', {'field': 'cusip', 'count': 1})
('Microsoft Corporation (MSFT)', {'field': 'issuer_name', 'count': 2})
('594918104', {'field': 'cusip', 'count': 4698})
('Microsoft Corp.', {'field': 'issuer_name', 'count': 60})
('MICROSOFT CORP COM', {'field': 'issuer_name', 'count': 1})
('GE Healthcare Technologies Inc', {'field': 'issuer_name', 'count': 9})
('36266G107', {'field': 'cusip', 'count': 549})
('Duke Energy Corp.', {'field': 'issuer_name', 

/var/folders/z7/gyrmrn1s10zf6k523_6hg_br0000gn/T/ipykernel_40657/942150125.py:18: UserWarning: 2 subgraphs have name='chevron'.
[len(subg.nodes) for subg in collected]=[17, 1]
  warn(alert_str)
/var/folders/z7/gyrmrn1s10zf6k523_6hg_br0000gn/T/ipykernel_40657/942150125.py:18: UserWarning: 2 subgraphs have name='chevron'.
[len(subg.nodes) for subg in collected]=[28, 1]
  warn(alert_str)
/var/folders/z7/gyrmrn1s10zf6k523_6hg_br0000gn/T/ipykernel_40657/942150125.py:18: UserWarning: 2 subgraphs have name='jpmorgan'.
[len(subg.nodes) for subg in collected]=[25, 1]
  warn(alert_str)
/var/folders/z7/gyrmrn1s10zf6k523_6hg_br0000gn/T/ipykernel_40657/942150125.py:18: UserWarning: 2 subgraphs have name='jpmorgan'.
[len(subg.nodes) for subg in collected]=[1, 61]
  warn(alert_str)


In [8]:
msft_subgraph = extract_specific_name_subgraph(subgraphs, "microsoft")
mcds_subgraph = extract_specific_name_subgraph(subgraphs, "mcdonald")
jpm_subgraph = extract_specific_name_subgraph(subgraphs, "jpmorgan")

/var/folders/z7/gyrmrn1s10zf6k523_6hg_br0000gn/T/ipykernel_26279/942150125.py:18: UserWarning: 3 subgraphs have name='mcdonald'.
[len(subg.nodes) for subg in collected]=[1, 2, 54]
  warn(alert_str)
/var/folders/z7/gyrmrn1s10zf6k523_6hg_br0000gn/T/ipykernel_26279/942150125.py:18: UserWarning: 2 subgraphs have name='jpmorgan'.
[len(subg.nodes) for subg in collected]=[1, 61]
  warn(alert_str)


In [9]:
ps(msft_subgraph)

('GE HealthCare Technologies Inc', {'field': 'issuer_name', 'count': 1})
('GE Healthcare Technologies', {'field': 'issuer_name', 'count': 2})
('GE HEALTHCARE TECHNOLOGIES INC', {'field': 'issuer_name', 'count': 2})
('MICROSOFT CORPORATION', {'field': 'issuer_name', 'count': 2555})
('DUKE ENERGY CORPORATION', {'field': 'issuer_name', 'count': 768})
('BBG000BPH459', {'field': 'figi', 'count': 79})
('Duke Energy', {'field': 'issuer_name', 'count': 4})
('594918103', {'field': 'cusip', 'count': 3})
('DUKE ENERGYCORPORATION', {'field': 'issuer_name', 'count': 1})
('DUKE ENERGY\nCORPORATION', {'field': 'issuer_name', 'count': 39})
('Microsoft Annual Meeting', {'field': 'issuer_name', 'count': 1})
('GE Healthcare Tech', {'field': 'issuer_name', 'count': 1})
('Mckesson Corp', {'field': 'issuer_name', 'count': 1})
('Microsoft', {'field': 'issuer_name', 'count': 6})
('26441c204', {'field': 'cusip', 'count': 1})
('BBG001S5TD05', {'field': 'figi', 'count': 1})
('USU3644QAF29', {'field': 'isin', 'co